In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.preprocessing import image

import cv2
import numpy as np

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dense(4, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/My Drive/data-comvi/dataset',
    target_size=(150, 150),
    batch_size=20,
    class_mode='categorical'
)

history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=20
)

Found 2010 images belonging to 4 classes.
Epoch 1/20
100/100 [==============================] - 431s 4s/step - loss: 1.1522 - accuracy: 0.4945
Epoch 2/20
 38/100 [==========>...................] - ETA: 1:01 - loss: 0.7091 - accuracy: 0.7079

KeyboardInterrupt: 

In [ ]:
# Save โมเดล
model.save('/content/drive/My Drive/saved_model/my_model')

# Load โมเดล
loaded_model = tf.keras.models.load_model('/content/drive/My Drive/saved_model/my_model')

In [ ]:
# ทำการทดสอบโมเดลโหลดมา
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    '/content/drive/My Drive/data-comvi/test',
    target_size=(150, 150),
    batch_size=20,
    class_mode='categorical'
)

test_loss, test_acc = loaded_model.evaluate(test_generator)
print('Test loss: ', test_loss)
print('Test accuracy: ', test_acc)

In [ ]:
# พล็อตกราฟค่าความแม่นยำของโมเดลในชุดข้อมูลฝึกและทดสอบ
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training Accuracy')
plt.legend()
plt.show()

print(" ")

# พล็อตกราฟค่าความสูญเสียของโมเดลในชุดข้อมูลฝึกและทดสอบ
plt.plot(history.history['loss'], label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.legend()
plt.show()

In [ ]:
# โหลดรูปภาพ
image_path = '/content/drive/My Drive/test-comvi/chair5.jpg'  # แทนที่ด้วยที่อยู่ของรูปภาพที่คุณต้องการทดสอบ
image = cv2.imread(image_path)
resized_image = cv2.resize(image, (150, 150))  # ปรับขนาดรูปภาพให้เหมาะสมกับโมเดล (150x150 pixels)

# ทำนายคลาสของรูปภาพ
normalized_image = resized_image / 255.0  # ทำการ normalize รูปภาพ
input_image = np.expand_dims(normalized_image, axis=0)  # เพิ่มมิติเพื่อให้มีรูปแบบ batch
prediction = loaded_model.predict(input_image)

# แปลงผลลัพธ์เป็นชื่อคลาส
class_names = ['bed', 'chair', 'door', 'lamp']  # รายชื่อคลาส
predicted_class_index = np.argmax(prediction)
predicted_class_name = class_names[predicted_class_index]

# แสดงผลลัพธ์
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.show()
print(" ")
print("Predicted Class : ", predicted_class_name)